In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import gc
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso, Ridge

In [3]:
items           = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train_v2.csv')
test            = pd.read_csv('test.csv')
sample_submission            = pd.read_csv('sample_submission.csv')

In [4]:
transactions = sales_train.copy()
transactions[['day','month', 'year']] = transactions['date'].str.split('.', expand=True).astype(int)


In [5]:
w_cat_ids = transactions.set_index('item_id').join(items.set_index('item_id'))
w_cat_ids = w_cat_ids.reset_index()
w_cat_ids['row_id'] = w_cat_ids.index
w_cat_ids['y'] = w_cat_ids.groupby(['date_block_num', 'item_id', 'shop_id'])['item_cnt_day'].transform('sum').clip(0,20)
w_cat_ids = w_cat_ids.drop_duplicates(['date_block_num', 'item_id', 'shop_id'])
w_cat_ids = w_cat_ids[w_cat_ids['year'] != 2013]


In [6]:
w_cat_ids['item_mean'] = w_cat_ids.groupby(['item_id'])['y'].transform('mean')
w_cat_ids['item_mean_month'] = w_cat_ids.groupby(['item_id', 'month'])['y'].transform('mean')
w_cat_ids['shop_mean_month'] = w_cat_ids.groupby(['shop_id', 'month'])['y'].transform('mean')
w_cat_ids['category_mean_month'] = w_cat_ids.groupby(['item_category_id', 'month'])['y'].transform('mean')
w_cat_ids['shop_category_mean_month'] = w_cat_ids.groupby(['shop_id','item_category_id', 'month'])['y'].transform('mean')

In [7]:
X = w_cat_ids[(w_cat_ids['year'] == 2015) | ((w_cat_ids['year'] == 2014) & (w_cat_ids['month'].isin([11,12])))]

In [8]:
rollings = X[['item_mean_month', 'date_block_num', 'item_id']].copy()
rollings = rollings.drop_duplicates(['item_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_id'])
a = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(1)
aa = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(3)
aaa = rollings.sort_values(['item_id', 'date_block_num'])\
        .groupby(['item_id'])['item_mean_month'].shift(6)

In [9]:
b = pd.DataFrame(a)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [10]:
b = pd.DataFrame(aa)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [11]:
b = pd.DataFrame(aaa)
b.rename(columns={'item_mean_month': 'item_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_id', 'date_block_num'], how='left')

In [12]:
rollings = X[['shop_mean_month', 'date_block_num', 'shop_id']].copy()
rollings = rollings.drop_duplicates(['shop_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'shop_id'])
a = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(1)
aa = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(3)
aaa = rollings.sort_values(['shop_id', 'date_block_num'])\
        .groupby(['shop_id'])['shop_mean_month'].shift(6)

In [13]:
b = pd.DataFrame(a)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [14]:
b = pd.DataFrame(aa)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [15]:
b = pd.DataFrame(aaa)
b.rename(columns={'shop_mean_month': 'shop_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['shop_id', 'date_block_num'], how='left')

In [16]:
rollings = X[['category_mean_month', 'date_block_num', 'item_category_id']].copy()
rollings = rollings.drop_duplicates(['item_category_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_category_id'])
a = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(1)
aa = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(3)
aaa = rollings.sort_values(['item_category_id', 'date_block_num'])\
        .groupby(['item_category_id'])['category_mean_month'].shift(6)

In [17]:
b = pd.DataFrame(a)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [18]:
b = pd.DataFrame(aa)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [19]:
b = pd.DataFrame(aaa)
b.rename(columns={'category_mean_month': 'category_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'date_block_num'], how='left')

In [20]:
rollings = X[['shop_category_mean_month', 'date_block_num', 'shop_id', 'item_category_id']].copy()
rollings = rollings.drop_duplicates(['item_category_id', 'shop_id', 'date_block_num'])
rollings = rollings.set_index(['date_block_num', 'item_category_id', 'shop_id'])
a = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(1)
aa = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(3)
aaa = rollings.sort_values(['item_category_id', 'shop_id', 'date_block_num'])\
        .groupby(['item_category_id', 'shop_id'])['shop_category_mean_month'].shift(6)

In [21]:
b = pd.DataFrame(a)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_1'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [22]:
b = pd.DataFrame(aa)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_3'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [23]:
b = pd.DataFrame(aaa)
b.rename(columns={'shop_category_mean_month': 'shop_category_mean_month_minus_6'}, inplace=True)

X = pd.merge(X, b, on=['item_category_id', 'shop_id', 'date_block_num'], how='left')

In [24]:
X[X['item_id'] == 30].drop_duplicates(['item_id', 'date_block_num'])[['item_id', 'month', 'year', 'item_mean_month', 'item_mean_month_minus_1', 'item_mean_month_minus_3', 'item_mean_month_minus_6']]

,item_id,month,year,item_mean_month,item_mean_month_minus_1,item_mean_month_minus_3,item_mean_month_minus_6
10,30,11,2014,1.300000,NaN,NaN,NaN
20,30,12,2014,1.600000,1.300000,NaN,NaN
30,30,1,2015,1.894737,1.600000,NaN,NaN
41,30,2,2015,1.266667,1.894737,1.300000,NaN
53,30,3,2015,1.714286,1.266667,1.600000,NaN
58,30,4,2015,1.388889,1.714286,1.894737,NaN
62,30,5,2015,1.235294,1.388889,1.266667,1.300000
66,30,6,2015,1.133333,1.235294,1.714286,1.600000
70,30,7,2015,1.214286,1.133333,1.388889,1.894737
73,30,8,2015,1.285714,1.214286,1.235294,1.266667


In [25]:
X[X['shop_id'] == 30].drop_duplicates(['shop_id', 'date_block_num'])[['shop_id', 'month', 'year', 'shop_mean_month', 'shop_mean_month_minus_1', 'shop_mean_month_minus_3', 'shop_mean_month_minus_6']]

,shop_id,month,year,shop_mean_month,shop_mean_month_minus_1,shop_mean_month_minus_3,shop_mean_month_minus_6
11,30,11,2014,1.873958,NaN,NaN,NaN
35,30,1,2015,1.736099,2.118103,NaN,NaN
130,30,2,2015,1.670232,1.736099,1.873958,NaN
245,30,12,2014,2.118103,1.873958,NaN,NaN


In [26]:
X[X['item_category_id'] == 30].drop_duplicates(['item_category_id', 'date_block_num'])[['item_category_id', 'month', 'year', 'category_mean_month', 'category_mean_month_minus_1']]

,item_category_id,month,year,category_mean_month,category_mean_month_minus_1
3762,30,10,2015,3.070205,2.746511
11027,30,1,2015,3.009862,3.101948
11028,30,7,2015,2.670056,2.788012
12837,30,11,2014,2.843284,NaN
12854,30,12,2014,3.101948,2.843284
12897,30,2,2015,2.905608,3.009862
12904,30,3,2015,2.889822,2.905608
12913,30,4,2015,2.892322,2.889822
12925,30,5,2015,2.702312,2.892322
12933,30,6,2015,2.788012,2.702312


In [27]:
X[(X['item_category_id'] == 30) & (X['shop_id'] == 30)].drop_duplicates(['item_category_id', 'shop_id', 'date_block_num'])[['item_category_id', 'shop_id' ,  'month', 'year', 'shop_category_mean_month', 'shop_category_mean_month_minus_1']]

,item_category_id,shop_id,month,year,shop_category_mean_month,shop_category_mean_month_minus_1
16939,30,30,11,2014,1.864198,NaN
16992,30,30,12,2014,2.421687,1.864198
17043,30,30,1,2015,2.597633,2.421687
17087,30,30,2,2015,2.488550,2.597633


In [28]:
X.columns

Index(['item_id', 'date', 'date_block_num', 'shop_id', 'item_price',
       'item_cnt_day', 'day', 'month', 'year', 'item_name', 'item_category_id',
       'row_id', 'y', 'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 'item_mean_month_minus_3',
       'item_mean_month_minus_6', 'shop_mean_month_minus_1',
       'shop_mean_month_minus_3', 'shop_mean_month_minus_6',
       'category_mean_month_minus_1', 'category_mean_month_minus_3',
       'category_mean_month_minus_6', 'shop_category_mean_month_minus_1',
       'shop_category_mean_month_minus_3', 'shop_category_mean_month_minus_6'],
      dtype='object')

In [29]:
columns = X.columns
percent_missing = X.isna().sum() * 100 / len(X)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

,column_name,percent_missing
item_id,item_id,0.000000
date,date,0.000000
date_block_num,date_block_num,0.000000
shop_id,shop_id,0.000000
item_price,item_price,0.000000
item_cnt_day,item_cnt_day,0.000000
day,day,0.000000
month,month,0.000000
year,year,0.000000
item_name,item_name,0.000000


In [30]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['item_id'] = test_w_cat_ids.index

In [31]:
#features = ['item_id', 'date_block_num', 'shop_id', #'item_price',
lgbm_features = ['item_id', 'shop_id',  'month', 'item_category_id',
       'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 'item_mean_month_minus_3',
       'item_mean_month_minus_6', 'shop_mean_month_minus_1',
       'shop_mean_month_minus_3', 'shop_mean_month_minus_6',
       'category_mean_month_minus_1', 'category_mean_month_minus_3',
       'category_mean_month_minus_6', 'shop_category_mean_month_minus_1',
       'shop_category_mean_month_minus_3', 'shop_category_mean_month_minus_6']
    
lr_features = [
       'item_mean', 'item_mean_month', 'shop_mean_month',
       'category_mean_month', 'shop_category_mean_month',
       'item_mean_month_minus_1', 
       'shop_mean_month_minus_1',
       'category_mean_month_minus_1',
       'shop_category_mean_month_minus_1',]


In [32]:
val_indices = X[(X['year'] == 2015) & (X['month'].isin([9,10]))].index

In [33]:
x = X
y = X['y']


x_train = x[~x.index.isin(val_indices)]
y_train = y[~y.index.isin(val_indices)]

x_val = x[x.index.isin(val_indices)]
y_val = y[y.index.isin(val_indices)]

In [34]:
lgtrain = lgbm.Dataset(x_train[lgbm_features], label=y_train, categorical_feature=[0,1,2,3])
lgval = lgbm.Dataset(x_val[lgbm_features], label=y_val, categorical_feature=[0,1,2,3])



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,
        #"num_leaves": 10,
        "bagging_fraction": 0.3,
        "bagging_freq": 1,
        "min_data_in_leaf": 5000,
        "feature_fraction": 0.5,
        #"lambda_l2": 3,
        #"max_depth": 2,
        "learning_rate" : 0.01
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 50000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=500, 
                      evals_result=evals_result)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.81865
[1000]	valid_0's rmse: 1.73614
[1500]	valid_0's rmse: 1.69838
[2000]	valid_0's rmse: 1.67381
[2500]	valid_0's rmse: 1.65572
[3000]	valid_0's rmse: 1.64298
[3500]	valid_0's rmse: 1.63354
[4000]	valid_0's rmse: 1.62574


KeyboardInterrupt: 

In [92]:
x_train_lr = x[~x.index.isin(val_indices)].copy()[lr_features]
x_train_lr.fillna(x_train_lr.mean(), inplace=True)

x_val_lr = x[x.index.isin(val_indices)].copy()[lr_features]
x_val_lr.fillna(x_train_lr.mean(), inplace=True)


In [93]:
lr_model = Ridge(alpha=0.5)
lr_model.fit(x_train_lr, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [95]:
from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_lr)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.6796097341448792


In [90]:
train_meta['date_block_num'].unique()

array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [98]:
#stacking
blocks = [28, 29, 30, 31, 32, 33]

train_meta = X.copy()
train_meta = train_meta.sort_values(by=['date_block_num']).reset_index()

lr_preds = np.zeros(len(train_meta))


for block in blocks:
    
    fold = train_meta[train_meta['date_block_num'] == block]
    train = train_meta[train_meta['date_block_num'] < block]
    y = train['y']
    
    #lr
    x_lr = train[lr_features].copy()
    x_lr.fillna(x_lr.mean(), inplace=True)
    
    lr_model = Ridge(alpha=0.5)
    lr_model.fit(x_lr, y)
    oof = fold[lr_features].copy()
    oof.fillna(oof.mean(), inplace=True)
    oof_pred = lr_model.predict(oof)
    oof_pred.clip(0,20,out=oof_pred)

    lr_preds[fold.index] = oof_pred
    
train_meta['lr_pred'] = lr_preds




In [104]:
lgbm_preds = np.zeros(len(train_meta))

for block in blocks:
    
    fold = train_meta[train_meta['date_block_num'] == block]
    train = train_meta[train_meta['date_block_num'] < block]
    y = train['y']
    
    #lgbm
    val_len = int(10 * len(train) / 100)
    print(val_len)
    val_indices = sorted(train.index)[-val_len:]


    x_train = train[~train.index.isin(val_indices)]
    y_train = y[~y.index.isin(val_indices)]

    x_val = train[train.index.isin(val_indices)]
    y_val = y[y.index.isin(val_indices)]
    
    
    
    lgtrain = lgbm.Dataset(x_train[lgbm_features], label=y_train, categorical_feature=[0,1,2,3])
    lgval = lgbm.Dataset(x_val[lgbm_features], label=y_val, categorical_feature=[0,1,2,3])
    
    
    
    params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        #"max_bin": 10,
        #"num_leaves": 10,
        "bagging_fraction": 0.3,
        "bagging_freq": 1,
        "min_data_in_leaf": 5000,
        "feature_fraction": 0.5,
        #"lambda_l2": 3,
        #"max_depth": 2,
        "learning_rate" : 0.01
    }

    evals_result = {}
    model_lgb = lgbm.train(params, lgtrain, 50000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=500, 
                      evals_result=evals_result)
    
    oof = fold[lgbm_features]

    oof_preds = model_lgb.predict(oof)
    oof_preds.clip(0,20,out=oof_preds)

    lgbm_preds[fold.index] = oof_preds
    
train_meta['lgbm_pred'] = lgbm_preds

import pickle


26653


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.92042
[1000]	valid_0's rmse: 1.85143
[1500]	valid_0's rmse: 1.8128
[2000]	valid_0's rmse: 1.79155
[2500]	valid_0's rmse: 1.77334
[3000]	valid_0's rmse: 1.76044
[3500]	valid_0's rmse: 1.74919
[4000]	valid_0's rmse: 1.74005
[4500]	valid_0's rmse: 1.733
[5000]	valid_0's rmse: 1.72626
[5500]	valid_0's rmse: 1.72051
[6000]	valid_0's rmse: 1.71578
[6500]	valid_0's rmse: 1.71186
[7000]	valid_0's rmse: 1.70756
[7500]	valid_0's rmse: 1.70392
[8000]	valid_0's rmse: 1.70124
[8500]	valid_0's rmse: 1.69761
[9000]	valid_0's rmse: 1.69429
[9500]	valid_0's rmse: 1.69166
[10000]	valid_0's rmse: 1.68907
[10500]	valid_0's rmse: 1.68709
[11000]	valid_0's rmse: 1.68503
[11500]	valid_0's rmse: 1.68203
[12000]	valid_0's rmse: 1.68022
[12500]	valid_0's rmse: 1.67811
[13000]	valid_0's rmse: 1.67641
[13500]	valid_0's rmse: 1.67416
[14000]	valid_0's rmse: 1.67208
[14500]	valid_0's rmse: 1.67022
[15000]	valid_0's rmse: 1.66864


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.79144
[1000]	valid_0's rmse: 1.7112
[1500]	valid_0's rmse: 1.66996
[2000]	valid_0's rmse: 1.64124
[2500]	valid_0's rmse: 1.62209
[3000]	valid_0's rmse: 1.60648
[3500]	valid_0's rmse: 1.59446
[4000]	valid_0's rmse: 1.58459
[4500]	valid_0's rmse: 1.57622
[5000]	valid_0's rmse: 1.57014
[5500]	valid_0's rmse: 1.56415
[6000]	valid_0's rmse: 1.55914
[6500]	valid_0's rmse: 1.55442
[7000]	valid_0's rmse: 1.55013
[7500]	valid_0's rmse: 1.54647
[8000]	valid_0's rmse: 1.54352
[8500]	valid_0's rmse: 1.54028
[9000]	valid_0's rmse: 1.53732
[9500]	valid_0's rmse: 1.53457
[10000]	valid_0's rmse: 1.53231
[10500]	valid_0's rmse: 1.52972
[11000]	valid_0's rmse: 1.52723
[11500]	valid_0's rmse: 1.52513
[12000]	valid_0's rmse: 1.52345
[12500]	valid_0's rmse: 1.52133
[13000]	valid_0's rmse: 1.51915
[13500]	valid_0's rmse: 1.51715
[14000]	valid_0's rmse: 1.51582
[14500]	valid_0's rmse: 1.51407
[15000]	valid_0's rmse: 1.5127

...

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.51077
[1000]	valid_0's rmse: 1.44267
[1500]	valid_0's rmse: 1.40808
[2000]	valid_0's rmse: 1.38562
[2500]	valid_0's rmse: 1.36929
[3000]	valid_0's rmse: 1.35844
[3500]	valid_0's rmse: 1.34996
[4000]	valid_0's rmse: 1.34341
[4500]	valid_0's rmse: 1.33774
[5000]	valid_0's rmse: 1.33366
[5500]	valid_0's rmse: 1.33001
[6000]	valid_0's rmse: 1.32725
[6500]	valid_0's rmse: 1.32437
[7000]	valid_0's rmse: 1.32185
[7500]	valid_0's rmse: 1.31979
[8000]	valid_0's rmse: 1.31787
[8500]	valid_0's rmse: 1.31618
[9000]	valid_0's rmse: 1.31501
[9500]	valid_0's rmse: 1.31313
[10000]	valid_0's rmse: 1.31182
Early stopping, best iteration is:
[10389]	valid_0's rmse: 1.31085
39767


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.49355
[1000]	valid_0's rmse: 1.43757
[1500]	valid_0's rmse: 1.41135
[2000]	valid_0's rmse: 1.39347
[2500]	valid_0's rmse: 1.38128
[3000]	valid_0's rmse: 1.37304
[3500]	valid_0's rmse: 1.36693
[4000]	valid_0's rmse: 1.36201
[4500]	valid_0's rmse: 1.35836
[5000]	valid_0's rmse: 1.35546
[5500]	valid_0's rmse: 1.35264
[6000]	valid_0's rmse: 1.35052
[6500]	valid_0's rmse: 1.34821
[7000]	valid_0's rmse: 1.34633
[7500]	valid_0's rmse: 1.34493
[8000]	valid_0's rmse: 1.34359
[8500]	valid_0's rmse: 1.3421
[9000]	valid_0's rmse: 1.34085
[9500]	valid_0's rmse: 1.33945
[10000]	valid_0's rmse: 1.33813
[10500]	valid_0's rmse: 1.33722
[11000]	valid_0's rmse: 1.33651
[11500]	valid_0's rmse: 1.33558
[12000]	valid_0's rmse: 1.33468
[12500]	valid_0's rmse: 1.33384
Early stopping, best iteration is:
[12869]	valid_0's rmse: 1.33306
42735


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's rmse: 1.70263
[1000]	valid_0's rmse: 1.63411
[1500]	valid_0's rmse: 1.60018
[2000]	valid_0's rmse: 1.57759
[2500]	valid_0's rmse: 1.56215
[3000]	valid_0's rmse: 1.55133
[3500]	valid_0's rmse: 1.54281
[4000]	valid_0's rmse: 1.5363
[4500]	valid_0's rmse: 1.53026
[5000]	valid_0's rmse: 1.52641
[5500]	valid_0's rmse: 1.52258
[6000]	valid_0's rmse: 1.52006
[6500]	valid_0's rmse: 1.51685
[7000]	valid_0's rmse: 1.51499
[7500]	valid_0's rmse: 1.5124
[8000]	valid_0's rmse: 1.51063
[8500]	valid_0's rmse: 1.50877
[9000]	valid_0's rmse: 1.50738
[9500]	valid_0's rmse: 1.50551
[10000]	valid_0's rmse: 1.50387
[10500]	valid_0's rmse: 1.5027
[11000]	valid_0's rmse: 1.5012
[11500]	valid_0's rmse: 1.5003
[12000]	valid_0's rmse: 1.49931
[12500]	valid_0's rmse: 1.49848
[13000]	valid_0's rmse: 1.4973
[13500]	valid_0's rmse: 1.49643
[14000]	valid_0's rmse: 1.49589
[14500]	valid_0's rmse: 1.49507
Early stopping, best iteration is:


In [102]:
lgbm_preds.shape

(32220,)

In [73]:
train_meta['lr_preds'].head()

0    0.896845
1    1.721766
2    1.946306
3    2.229214
4    2.253534
Name: lr_preds, dtype: float64

In [ ]:
test_w_cat_ids = test.set_index('item_id').join(items.set_index('item_id'))
test_w_cat_ids['item_id'] = test_w_cat_ids.index

In [ ]:
test_w_cat_ids['month'] = 11

In [ ]:
item_level = X.drop_duplicates(['item_id'])[['item_id','item_mean', 'item_price']]

test_w_cat_ids = pd.merge(test_w_cat_ids, item_level, on=['item_id'], how='left')

In [ ]:
item_month_level = X[(X.month == 11)].drop_duplicates(['item_id'])[['item_id','item_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level, on=['item_id'], how='left')

In [ ]:
item_month_level_ = X[X.month == 10].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [ ]:
item_month_level_ = X[X.month == 8].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [ ]:
item_month_level_ = X[X.month == 5].drop_duplicates(['item_id'])[['item_id','item_mean_month']]\
                                   .copy().rename(columns={'item_mean_month':'item_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, item_month_level_, on=['item_id'], how='left')

In [ ]:
shop_month_level_ = X[(X.month == 11)].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [ ]:
shop_month_level_ = X[X.month == 10].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [ ]:
shop_month_level_ = X[X.month == 8].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [ ]:
shop_month_level_ = X[X.month == 5].drop_duplicates(['shop_id'])[['shop_id','shop_mean_month']]\
                                   .copy().rename(columns={'shop_mean_month':'shop_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_month_level_, on=['shop_id'], how='left')

In [ ]:
cat_month_level_ = X[(X.month == 11)].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [ ]:
cat_month_level_ = X[X.month == 10].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [ ]:
cat_month_level_ = X[X.month == 8].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [ ]:
cat_month_level_ = X[X.month == 5].drop_duplicates(['item_category_id'])[['item_category_id','category_mean_month']]\
                                   .copy().rename(columns={'category_mean_month':'category_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, cat_month_level_, on=['item_category_id'], how='left')

In [ ]:
shop_cat_month_level_ = X[(X.month == 11)].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]

test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [ ]:
shop_cat_month_level_ = X[X.month == 10].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_1'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [ ]:
shop_cat_month_level_ = X[X.month == 8].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_3'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [ ]:
shop_cat_month_level_ = X[X.month == 5].drop_duplicates(['item_category_id', 'shop_id'])[['item_category_id', 'shop_id', 'shop_category_mean_month']]\
                                   .copy().rename(columns={'shop_category_mean_month':'shop_category_mean_month_minus_6'})


test_w_cat_ids = pd.merge(test_w_cat_ids, shop_cat_month_level_, on=['item_category_id', 'shop_id'], how='left')

In [ ]:
test_w_cat_ids[features].head()

In [ ]:
columns = X.columns
percent_missing = X.isna().sum() * 100 / len(X)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

In [ ]:
columns = test_w_cat_ids.columns
percent_missing = test_w_cat_ids.isna().sum() * 100 / len(test_w_cat_ids)
pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

In [ ]:
lr_preds = lr_model.predict(test_w_cat_ids[lr_features])

In [ ]:
preds = model_lgb.predict(test_w_cat_ids[features])
preds.clip(0,20,out=preds)

In [ ]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds.astype(int)

submission.to_csv('submission.csv', index=False)

In [ ]:
print('Plotting feature importances...')
ax = lgbm.plot_importance(model_lgb, max_num_features=20)
plt.show()

3.45 val > 3.70 submit
3.06 val > 3.65 submit